In [1]:
import json
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import mplhep as mh

from hist.hist import Hist
from hist.axis import Regular

from pathlib import Path

from typing import Optional, Union

In [2]:
from functools import cache
from functools import cached_property

@cache
def get_segment(ring: int, station: int, sector: int, subsector: int) -> int:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L361-L368
    """
    nsub = 3 if ring == 1 and station > 1 else 6
    return subsector + nsub * (sector - 1)


@cache
def get_roll_name(region: int, ring: int, station: int, sector: int, layer: int,
             subsector: int, roll: int
) -> str:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L11-L87
    """
    if region == 0:
        name = f'W{ring:+d}_RB{station}'

        if station <= 2:
            name += 'in' if layer == 1 else 'out'
        else:
            if sector == 4 and station == 4:
                name += ['--', '-', '+', '++'][subsector - 1]
            elif (station == 3) or (station == 4 and sector not in (4, 9, 11)):
                name += '-' if subsector == 1 else '+'
        name += f'_S{sector:0>2d}_'
        name += ['Backward', 'Middle', 'Forward'][roll - 1]
    else:
        segment = get_segment(ring, station, sector, subsector)
        name = f'RE{station * region:+d}_R{ring}_CH{segment:0>2d}_'
        name += ['A', 'B', 'C', 'D', 'E'][roll - 1]
    return name

In [32]:
all_columns = [
    'is_fiducial', 'is_matched', 
    'region', 'ring', 'station', 'sector', 'layer', 'subsector', 'roll', 
    'run', 'cls', 'bx', 'event'
    'tag_pt', 'tag_eta', 'tag_phi', 
    'probe_pt', 'probe_eta', 'probe_phi', 'probe_time', 'probe_dxdz', 'probe_dydz', 
    'dimuon_pt', 'dimuon_mass', 
    'residual_x', 'residual_y', 'pull_x', 'pull_y', 'pull_x_v2', 'pull_y_v2', 
]

columns = [
    'is_fiducial', 'is_matched', 
    'region', 'ring', 'station', 'sector', 'layer', 'subsector', 'roll', 
    'cls', 'bx', 'run', 'event'
] 

In [50]:
working_dir = Path('/users/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting')
input_path = working_dir / 'data' / 'Muon__Run2022C.root'

data = uproot.open(f"{str(input_path)}:tree").arrays(columns, library='pd')

data['segment'] = data.apply(lambda row: get_segment(row['ring'], row['station'], row['sector'], row['subsector']), axis=1)

data['roll_name'] = data.apply(lambda row: get_roll_name(
    row['region'], row['ring'], row['station'],
    row['sector'], row['layer'], row['subsector'], row['roll']
), axis=1)

In [46]:
for idx in range(data.size):
    print(data.)
    break

is_fiducial


In [45]:
print(data.columns)

name_arr = [get_roll_name(row.region, row.ring, row.station,
                          row.sector, row.layer, row.subsector,
                          row.roll)
            for row in data]
name_arr = np.array(name_arr)

Index(['is_fiducial', 'is_matched', 'region', 'ring', 'station', 'sector',
       'layer', 'subsector', 'roll', 'cls', 'bx', 'run'],
      dtype='object')


AttributeError: 'str' object has no attribute 'region'